In [ ]:
pip install --upgrade azureml-sdk

In [ ]:
pip install --upgrade dask-cloudprovider

**RESTART YOUR KERNEL**

## Workspace

Get the Azure ML workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config('~/code/config.json')
ws

In [ ]:
from cryptography.hazmat.primitives import serialization as crypto_serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend as crypto_default_backend

key = rsa.generate_private_key(
    backend=crypto_default_backend(),
    public_exponent=65537,
    key_size=2048
)
private_key = key.private_bytes(
    crypto_serialization.Encoding.PEM,
    crypto_serialization.PrivateFormat.PKCS8,
    crypto_serialization.NoEncryption())
public_key = key.public_key().public_bytes(
    crypto_serialization.Encoding.OpenSSH,
    crypto_serialization.PublicFormat.OpenSSH
)

In [ ]:
import os

with open('key', 'wb') as f:
    f.write(private_key)
    os.chmod('key', 0o600)
with open('key.pub', 'wb') as f:
    f.write(public_key)

with open('key', 'r') as f:
    private_key = f.read()
with open('key.pub', 'r') as f:
    public_key = f.read()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
ct_name = 'dask-ssh-ct'

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                       min_nodes=1,
                                                       max_nodes=100,
                                                       admin_username='daskuser',
                                                       admin_user_ssh_key=public_key,
                                                       remote_login_port_public_access='Enabled'
)

ct = ComputeTarget.create(ws, ct_name, compute_config)
ct.wait_for_completion(show_output=True)

In [ ]:
from dask_cloudprovider import AzureMLCluster

cluster = AzureMLCluster(ws, 
                         compute_target=ct,
                         admin_username='daskuser',
                         admin_ssh_key='key',
                         scheduler_idle_timeout=7200, 
                         jupyter=True, 
                         show_output=True)

In [ ]:
cluster.run

In [ ]:
cluster.scale(20)

In [ ]:
cluster

In [ ]:
from dask.distributed import Client

c = Client(cluster)
c

In [ ]:
#cluster.close()